In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv('bank/bank-full.csv', sep=';')

# Select the required columns
columns = [
    "age", "job", "marital", "education", "balance", 
    "housing", "contact", "day", "month", "duration", 
    "campaign", "pdays", "previous", "poutcome", "y"
]
df = df[columns]


In [3]:
df.isnull().sum()


age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [13]:
education_mode = df['education'].mode()[0]
print(education_mode)


secondary


In [14]:
numerical_cols = ["age", "balance", "day", "duration", "campaign", "pdays", "previous"]
corr_matrix = df[numerical_cols].corr()

# Find the two features with the highest absolute correlation
corr_matrix_unstacked = corr_matrix.abs().unstack()
corr_matrix_unstacked = corr_matrix_unstacked[corr_matrix_unstacked < 1]  # Exclude self-correlation
highest_corr = corr_matrix_unstacked.idxmax()
print(highest_corr)


('pdays', 'previous')


In [6]:
df['y'] = df['y'].map({'yes': 1, 'no': 0})


In [7]:
from sklearn.model_selection import train_test_split

# Split data: 60% train, 20% validation, 20% test
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Separate the target variable
y_train = df_train.pop('y')
y_val = df_val.pop('y')
y_test = df_test.pop('y')


In [10]:
from sklearn.feature_selection import mutual_info_score

categorical_vars = ["job", "marital", "education", "contact", "housing", "poutcome"]
mutual_info_scores = {var: mutual_info_score(y_train, df_train[var]) for var in categorical_vars}

# Find the variable with the highest mutual information score
max_mutual_info_var = max(mutual_info_scores, key=mutual_info_scores.get)
print(max_mutual_info_var)


ImportError: cannot import name 'mutual_info_score' from 'sklearn.feature_selection' (/home/bench/Documents/projects/ML-zoomcamp-Homework/env/lib/python3.12/site-packages/sklearn/feature_selection/__init__.py)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define categorical columns for one-hot encoding
categorical_cols = ["job", "marital", "education", "contact", "housing", "poutcome", "month"]

# Create a pipeline for preprocessing and model training
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ],
    remainder='passthrough'
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42))
])

# Fit the model
model.fit(df_train, y_train)

# Predict and calculate accuracy on the validation set
y_val_pred = model.predict(df_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(round(val_accuracy, 2))


0.9


In [12]:
feature_importance = {}

for feature in df_train.columns:
    df_train_reduced = df_train.drop(columns=[feature])
    model.fit(df_train_reduced, y_train)
    y_val_pred = model.predict(df_val.drop(columns=[feature]))
    reduced_accuracy = accuracy_score(y_val, y_val_pred)
    feature_importance[feature] = val_accuracy - reduced_accuracy

# Find the feature with the smallest difference
least_impact_feature = min(feature_importance, key=feature_importance.get)
print(least_impact_feature)


ValueError: A given column is not a column of the dataframe